In [118]:
import pandas as pd
from datetime import date, timedelta
import altair as alt

In [119]:
# Configuration variables
today = date.today()
yesterday = date.today() - timedelta(days=1)
our_utla_codes = [
    'E08000025', # Birmingham
    'E08000027', # Dudley
    'E08000028', # Sandwell
    'E06000051', # Shropshire
    'E10000028', # Staffordshire
    'E06000021', # Stoke-on-Trent
    'E06000020', # Telford and Wrekin
    'E08000030', # Walsall
    'E08000031', # Wolverhampton
    'E10000034', # Worcestershire
]
pheh_url = 'https://fingertips.phe.org.uk/documents/Historic%20COVID-19%20Dashboard%20Data.xlsx'
pheh_header_row = 7 # zero indexed
pheh_utla_sheet = 'UTLAs'
pheh_utla_csv_filename = 'pheh-utla.csv'

In [120]:
# Import a specific sheet from the PHE Excel file and convert to CSV
pheh_utla_excel = pd.read_excel(pheh_url, header=pheh_header_row, sheet_name=pheh_utla_sheet)
pheh_utla_excel.to_csv(pheh_utla_csv_filename)
pheh_utla = pd.read_csv(pheh_utla_csv_filename)

In [121]:
our_utla = pheh_utla.rename(columns={"Area Code": "code", "Area Name" : "name"}) # clean up column names
our_utla = our_utla[our_utla.code.isin(our_utla_codes)] # filter to our trusts
our_utla.drop('Unnamed: 0', axis='columns', inplace=True)
our_utla = pd.melt(our_utla, id_vars=["code","name"], var_name="date", value_name="cases") # un-pivot the date columns
our_utla["date"] = pd.to_datetime(our_utla.date) # clean all dates to YYYY-MM-DD
our_utla["diff"] = our_utla.groupby("code")["cases"].diff().convert_dtypes(convert_integer=True) # add daily variance

In [122]:
reductions_in_cases = our_utla[our_utla["diff"] < 0] # instances where the cumulative case count went down
our_utlas_yesterday = our_utla[our_utla["date"] == pd.to_datetime(yesterday)]
our_utlas_today = our_utla[our_utla["date"] == pd.to_datetime(today)]

In [123]:
alt.Chart(our_utla).mark_line().encode(
    x='date',
    y='cases',
    color='name',
    tooltip='name'
)

alt.Chart(...)

In [124]:
alt.Chart(our_utla).mark_bar().encode(
    x='date',
    y='sum(cases)',
    color='name',
    tooltip='name'
)

alt.Chart(...)

In [125]:
alt.Chart(our_utla).mark_line().encode(
    x='date',
    y='diff',
    color='name',
    tooltip='name'
)

alt.Chart(...)

In [126]:
alt.Chart(our_utla).mark_bar().encode(
    x='date',
    y='sum(diff)',
    color='name',
    tooltip='name'
)

alt.Chart(...)

In [127]:
reductions_in_cases

,code,name,date,cases,diff
67,E08000031,Wolverhampton,2020-03-15,8,-1
72,E06000051,Shropshire,2020-03-16,2,-2
89,E10000034,Worcestershire,2020-03-17,1,-2


In [128]:
our_utlas_yesterday

,code,name,date,cases,diff
300,E06000020,Telford and Wrekin,2020-04-08,96,17
301,E06000021,Stoke-on-Trent,2020-04-08,130,13
302,E06000051,Shropshire,2020-04-08,174,17
303,E08000025,Birmingham,2020-04-08,1462,90
304,E08000027,Dudley,2020-04-08,341,32
305,E08000028,Sandwell,2020-04-08,399,36
306,E08000030,Walsall,2020-04-08,421,35
307,E08000031,Wolverhampton,2020-04-08,354,19
308,E10000028,Staffordshire,2020-04-08,673,76
309,E10000034,Worcestershire,2020-04-08,439,20


In [129]:
our_utlas_today

,code,name,date,cases,diff
